In [1]:
import h5py
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from glob import glob
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn
from torch.functional import F
from torch.optim import Adam
import numpy as np
from tqdm.auto import tqdm
import umap
import pandas as pd
from warnings import filterwarnings

In [2]:
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/'

In [ ]:
h5_files = glob(os.path.join(directory,'**/*.h5'))

In [ ]:
h5_files

In [ ]:
filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class MyH5Dataset(Dataset):
    
    def __init__(self,h5_paths):
        
        self.h5_paths  = h5_paths
        self._archives = [h5py.File(h5_path,"r") for h5_path in self.h5_paths]
        self.keys      = {}
        self.d = {}
        
        idx  = 0
        idx2 = 0
        
        
        for a, archive in enumerate(self.archives):
            
            for i in range(len(archive)):
                
                key = list(archive.keys())[i]  
                self.keys[idx] = (a, key)
                idx += 1
                
                for j in range(archive[key].shape[0]):
                    
                    self.d[idx2] = archive[key][j]
                    idx2 +=1
                    
        self._archive=None
    @property
    def archives(self):
        if self._archives is None: 
            self._archives = [h5py.File(h5_path, "r") for h5_path in self.h5_paths]
        return self._archives
    
    def __len__(self):
        return len(self.d)
    
    def __getitem__(self,index):
        
        data = self.d[index].astype(np.float32)
        data = torch.from_numpy(data)
        
        return data
        
    

In [ ]:
loader = torch.utils.data.DataLoader(MyH5Dataset(h5_files))

In [ ]:
doenum = enumerate(loader)

In [ ]:
idx,data = next(doenum)

# ConvVAE

In [ ]:
class ConvVAE(nn.Module):
    
    def __init__(self,latent_dim=200):
        super().__init__()
        self.encoder = nn.Sequential( 
            nn.Conv2d(in_channels=1,out_channels=8,stride=1,kernel_size=5), #320x320x1 --> 316x316x8
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=8,out_channels=16,stride=1,kernel_size=3), #316x316x8 --> 314x314x16
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16,out_channels=16,stride=2,kernel_size=4), #314x314x16 --> 156x156x16
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=16,out_channels=20,stride=1,kernel_size=3), #156x156x16 --> 154x154x20
            nn.LeakyReLU(),
        )
        
        
        self.mean    = nn.Linear(474320,latent_dim)
        self.log_var = nn.Linear(474320,latent_dim)
        self.linear  = nn.Linear(latent_dim,474320) 
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=20,out_channels=16,stride=1,kernel_size=3), # 154x154x20 --> 155x155x16
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=16,out_channels=16,stride=2,kernel_size=4), # 155x155x16 --> 312x312x16
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=16,out_channels=8,stride=1,kernel_size=4), # 312x312x16 --> 314x314x8
            nn.LeakyReLU(),
            nn.ConvTranspose2d(in_channels=8,out_channels=1,stride=1,kernel_size=7), # 314x314x8 --> 320x320x3
        )
        
    def reparameterize(self,z_mean,z_log_var):
        
        eps = torch.randn(z_mean.size(0),z_mean.size(1))
        z = z_mean + torch.exp(z_log_var/2) * eps
        
        return z
        
        
    def forward(self, x):
        
        x = self.encoder(x)
        print(x.shape)
        x = x.view(1,-1)
        z_mean, z_log_var = self.mean(x), self.log_var(x)
        
        
        encoded = self.reparameterize(self.mean(x), self.log_var(x))
        
        x = self.linear(encoded)
        
        x = x.view(1,20,154,154)
        
        decoded = self.decoder(x)
        
        
        return encoded, z_mean, z_log_var, decoded 

In [ ]:
def vae_loss(decoded,x,z_mean,z_log_var):
    
    x = x.view(x.size(0),-1)
    
    recons = F.mse_loss(decoded, x, reduction='sum')
    kl = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp())
    
    return recons + kl

In [ ]:
vae_model = ConvVAE()
optimizer = torch.optim.Adam(vae_model.parameters(), lr=3e-4)  

In [ ]:
def train(model,train_loader,optimizer,loss_fn,device,EPOCH):
    
    with tqdm(total=len(train_loader)*EPOCH*555) as tt:
        
        model.train()
        
        for epoch in range(EPOCH):
            
            total_loss, batch_count = 0, 0
            
            for idx, batch in enumerate(train_loader):
                
                batch = batch['data']
                batch = batch.view(batch.size(1),batch.size(0),batch.size(2),batch.size(3))
                
                for psf in batch:
                    
                    #print(batch.shape)
                    encoded, z_mean, z_log_var, reconstruction = model(psf)

                    loss = loss_fn(reconstruction,batch,z_mean,z_log_var)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    total_loss += loss.item()
                    batch_count +=1
                    tt.update()

                
            total_loss = total_loss / batch_count
            print(f'{total_loss}')
                
                

In [ ]:
train(model=vae_model,train_loader=loader,optimizer=optimizer,loss_fn=vae_loss,device=device,EPOCH=20)

In [ ]:
def visualize_resultsVAE(model,data_loader,data):
    dataiter = iter(data_loader)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    images, labels = dataiter.next()
    
    images_flatten = images.view(images.size(0), -1)
    _,_,_,output = model(images_flatten)
    
    images = images.numpy()
    output = output.view(32, 1, 28, 28)


    output = output.cpu().detach().numpy()

    fig, axes = plt.subplots(nrows=2, ncols=15, sharex=True, sharey=True, figsize=(25,4))

    for images, row in zip([images, output], axes):
        for img, ax in zip(images, row):
            
            ax.imshow((np.transpose(img,(1, 2, 0))*255).astype(np.uint8),cmap='gray')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

In [ ]:
visualize_resultsVAE(model=vae_model,data_loader=loader,data='fmnist')